In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [2]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

In [4]:
stats.describe(my_data)

DescribeResult(nobs=20, minmax=(0.006, 0.692), mean=0.1965, variance=0.027176368421052633, skewness=1.3751532772375619, kurtosis=2.0594082496235426)

In [6]:
mean = np.mean(my_data)
Lam = 1/(mean)
print(Lam)

5.089058524173028


In [9]:
ML = np.prod(Lam*e**(-Lam*my_data)) #one way to do it
print(ML)

279807.44620464457


In [11]:
HypLike = np.prod(3*e**(-3*my_data))
Lambda = HypLike/ML
print(Lambda)

0.09445694279678163


In [14]:
print('Test statistic of -2log(Lambda):',-2*log(Lambda))

Test statistic of -2log(Lambda): 4.719222360188457


In [15]:
p_value = stats.chi2.sf(-2*log(Lambda),1) #survival function (equivalent to 1-cdf)
p_value

0.02982722919477517

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [16]:
Crit = stats.chi2.ppf(.95,1)
Crit

3.841458820694124

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [20]:
Reject=[]
for i in np.arange(10000):
    Sample = stats.expon.rvs(size=20,scale=1/5)
    mu_sample = np.mean(Sample)
    lam_sample = 1/mu_sample
    Likelihood_3 = np.prod(3*e**(-3*Sample))
    ML = np.prod(lam_sample*e**(-lam_sample*Sample))
    Reject = np.append(Reject,-2*log(HypLike/ML)>=Crit)
np.mean(Reject)

0.5315

Repeat for a sample size of 50. What do you expect to happen to power? 

Since we are increasing the sample size we would expect power to increase

In [21]:
Reject=[]
for i in np.arange(10000):
    Sample = stats.expon.rvs(size=50,scale=1/5)
    mu_sample = np.mean(Sample)
    lam_sample = 1/mu_sample
    Likelihood_3 = np.prod(3*e**(-3*Sample))
    ML = np.prod(lam_sample*e**(-lam_sample*Sample))
    Reject = np.append(Reject,-2*log(HypLike/ML)>=Crit)
np.mean(Reject)

0.9971

### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

In [26]:
Simulated = np.array([np.mean(stats.expon.rvs(size=20,scale=1/3)) for _ in np.arange(10000)])
print(Simulated[1],Simulated[2],Simulated[3])
mu_data=np.mean(my_data)
print(mu_data)
p=np.mean((Simulated)<0.1965)+np.mean(Simulated>(.1965+1/3))
print(p)
print(np.mean(Simulated>(.1965+1/3)))

0.4297137871722513 0.27813383444368206 0.32844384787292497
0.1965
0.0264
0.0093


How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [27]:
Upper=percentile(.95,Simulated)
print(Upper)
lower=percentile(.05,Simulated)
print(lower)

0.18602283555572688
0.15047530607680448


Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [28]:
Simulated=np.array([np.mean(stats.expon.rvs(size=20,scale=1/3)) for _ in np.arange(10000)])
print(Simulated[1],Simulated[2],Simulated[3])

0.2994881805248979 0.23987372340835492 0.4729971251276847


Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [29]:
Simulated=np.array([np.mean(stats.expon.rvs(size=50,scale=1/3)) for _ in np.arange(10000)])
print(Simulated[1],Simulated[2],Simulated[3])

0.272091833478324 0.35623410121316856 0.3032114722181042
